# SQLAlchemy, Sqlite, and Dates

## Setup

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [7]:
engine = create_engine("sqlite:///../Resources/dow.sqlite", echo=False)

## Explore Database

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['dow']

In [9]:
dow_cols = inspector.get_columns('dow')
for col in dow_cols:
    print('{} {}'.format(col['name'], col['type']))

id INTEGER
quarter INTEGER
stock TEXT
date TEXT
open_price FLOAT
high_price FLOAT
low_price FLOAT
close_price FLOAT
volume INTEGER
percent_change FLOAT


In [10]:
engine.execute('SELECT * FROM dow LIMIT 5').fetchall()

[(1, 1, 'AA', '2011-01-07', 15.82, 16.72, 15.78, 16.42, 239655616, None),
 (2, 1, 'AA', '2011-01-14', 16.71, 16.71, 15.64, 15.97, 242963398, None),
 (3, 1, 'AA', '2011-01-21', 16.19, 16.38, 15.6, 15.79, 138428495, None),
 (4, 1, 'AA', '2011-01-28', 15.87, 16.63, 15.82, 16.13, 151379173, None),
 (5, 1, 'AA', '2011-02-04', 16.18, 17.39, 16.18, 17.14, 154387761, None)]

## Reflect and query dates

In [11]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [12]:
Dow = Base.classes.dow

In [13]:
session = Session(bind=engine)

### How many dates do we have?

In [14]:
session.query(Dow.date).count()

750

### What is the earliest date?

In [16]:
session.query(Dow.date).order_by(Dow.date).first()

('2011-01-07')

### What is the latest date?

In [17]:
session.query(Dow.date).order_by(Dow.date.desc()).first()

('2011-06-24')

### Find all of the dates greater than `2011-03-01`

In [19]:
session.query(Dow.date).filter(Dow.date > '2011-03-01').order_by(Dow.date).all()

[('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-04'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'),
 ('2011-03-11'

### Important Note! Sqlite does not support a date column type, but SQLAlchemy will allow you to work with dates in the iso format. [sqlite dates](http://docs.sqlalchemy.org/en/latest/dialects/sqlite.html)

In [20]:
import datetime as dt

In [21]:
# Print today's date 
print(dt.date.today())

2019-12-05


In [22]:
# print a specific date
print(dt.date(2018, 12, 20))

2018-12-20


In [23]:
# Print today's date using datetime
print(dt.datetime.utcnow())

2019-12-05 17:47:40.332045


In [24]:
# print a specific date using datetime
print(dt.datetime(2018, 12, 20))

2018-12-20 00:00:00


### Go back a week from a date by using time delta

In [26]:
print(dt.date.today() - dt.timedelta(days=7))

2019-11-28


### Go back a year from today by using time delta

In [30]:
print(dt.date.today() - dt.timedelta(days=365))

2018-12-05


### Query for the Dow closing price for stock 'CSCO' 1 week before `2011-04-08` using the datetime library

In [34]:
query_dt = dt.date(2011, 4, 8) - dt.timedelta(weeks=1)
session.query(Dow.date, Dow.close_price).\
        filter(Dow.stock == 'CSCO').\
        filter(Dow.date == query_dt).all()

[('2011-04-01', 17.04)]

### Parse out just the day from the datetime object

In [35]:
dt.date.today().strftime('%d')

'05'

### Query for all dates with day equal to 14
(matching the date string `14` in the format `%d`)

In [37]:
date_str = '14'
session.query(Dow.date).\
        filter(func.strftime('%d', Dow.date) == date_str).all()

[('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14'),
 ('2011-01-14')]